# Computing Observables

In [1]:
from set_fs import *
from observables import Autoload

In this exercise, you will be sampling from an already trained model and computing observables. A class `Autoload` has been created for you; it comes from the `observables` module. When instantiated, this object loads saved parameters from the `saved_params`. 

- It provides a `sample` method that can be used to sample from the network. By default, it would return 1000 samples but you can specify any batch size that you want.
- It also provides a method `logpsi` for computing the log of your amplitude

Note that the sample that is retured is a Jax array implementation. If you are not familiar with Jax, you can convert the output to numpy format by calling `numpy.asarray()` and passing the Jax array as input

In [2]:
# Sampling from the trained network using Autoload

nn_state = Autoload()
samples = nn_state.sample()
print(f"{samples.shape}, \n\n {samples}")

(1000, 16), 

 [[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 0]
 [1 1 0 ... 1 0 1]
 [0 1 0 ... 0 0 0]]


You can then decide to compute whatever observable that you are interested in. An example would be the average magnetization in the `z-direction` as shown below

In [3]:
from jax import numpy as jnp

In [4]:
# Start by coverting from (0, 1) to (-1, 1)
my_samples = nn_state.sample(500)
my_conv_samples =  2 * jnp.array(my_samples) - 1
print(my_conv_samples)

[[ 1  1 -1 ...  1 -1  1]
 [-1 -1 -1 ... -1 -1  1]
 [ 1 -1 -1 ...  1 -1 -1]
 ...
 [-1 -1 -1 ...  1 -1 -1]
 [ 1 -1  1 ... -1 -1  1]
 [-1 -1 -1 ... -1 -1  1]]


In [5]:
# Now, I can compute the average magnetization
jnp.mean(jnp.abs(jnp.mean(my_conv_samples, axis = 1))).item()

0.5097500085830688

However, I can create a class that inherits from `Observable` provided in the `observables` module. I will need to

- Compulsorily define a `compute` instance method in which I will define my implementation
- Optionally define my class name and symbol

Let us use the just completed magnetization as example. I can decide to create a class called `ZMagnetization`

In [6]:
from observables import Observable

class ZMagnetization(Observable):
    def __init__(self):
        self.name = "SigmaZ"
        self.symbol = "Z"

    def compute(self, model, sample_size):
        samps = model.sample(sample_size)
        conv_samples = 2 * jnp.array(samps) - 1

        ave_mag = jnp.mean(jnp.abs(jnp.mean(conv_samples, axis = 1)))

        return ave_mag.item()
        

Now, I can instantiate my class and ask it to compute the magnetization with the trained network using any sample size I fancy.

In [7]:
magnet_z = ZMagnetization()
magnet_z.compute(model = nn_state, sample_size = 2000)

0.5022500157356262

## YOUR TASKS - Compute Off-Diagonal Observables

1. Create a class that inherits `Observable` and can compute the expectation value of the in-plane magnetization - $\left < \sigma^x \right >$
2. Create a class that inherits `Observable` and can compute the second Renyi entropy $S_2$ as a function of the size of a sub-region $A$. It should be able to compute for detuning parameter 
    - Far away from criticality, and 
    - Close to criticality.